# Fruit recognition

## Chargement des modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as im
import sklearn as sk
import glob

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

## Chargement des données (train)

In [3]:
tabImg=pd.DataFrame()
categories = ['Apple Braeburn', 'Apple Granny Smith', 'Apricot', 'Avocado', 'Banana', 'Blueberry', 'Cactus fruit', 'Cantaloupe', 'Cherry', 'Clementine', 'Corn', 'Cucumber Ripe', 'Grape Blue', 'Kiwi', 'Lemon', 'Limes', 'Mango', 'Onion White', 'Orange', 'Papaya', 'Passion Fruit', 'Peach', 'Pear', 'Pepper Green', 'Pepper Red', 'Pineapple', 'Plum', 'Pomegranate', 'Potato Red', 'Raspberry', 'Strawberry', 'Tomato', 'Watermelon']
label = []
root_dir="data/train/"

for filename in glob.iglob(root_dir+'**/*.jpg',recursive=True):
    img=im.imread(filename)
    img=img.reshape(img.shape[0]*img.shape[1]*img.shape[2])
    
    fruit_name=filename[11:].split('/')[0]
    
    label.append(categories.index(fruit_name))
    tabImg=tabImg.append(pd.Series(img),ignore_index=True)
tabImg['label']=label

KeyboardInterrupt: 

In [ ]:
tabImg.shape

In [ ]:
tabImg.to_csv('train.csv',index=False)

In [ ]:
tabImg.head(5)

## Récupération des données (train)

In [ ]:
tab=pd.read_csv('train.csv')

In [ ]:
tab.shape

In [ ]:
tab.head(5)

In [ ]:
tab.isna().sum()

## Chargement des données (test)

In [ ]:
tabTest=pd.DataFrame()
import glob
root_dir="data/test/"
for filename in glob.iglob(root_dir+'**/*.jpg',recursive=True):
    img=im.imread(filename)
    img=img.reshape((1,img.shape[0]*img.shape[1]*img.shape[2]))
    
    new_row={'Image':img}
    tabTest=tabTest.append(new_row,ignore_index=True)

In [ ]:
tabTest.shape

In [ ]:
tabTest.head(5)

In [ ]:
tabTest.to_csv('test.csv',index=False)
np.save('test.npy', tabTest)

## Récupération des données (test)

In [ ]:
#npyTab = np.load('test.npy', allow_pickle=True)  # Ne rend pas un dataFrame
tabTestCsv=pd.read_csv('train.csv')

tabTestCsv.head(5)

In [ ]:
tabTestCsv.isna().sum()

# Traitement des donnée et modules

In [ ]:
X=tab.iloc[:,:-1]
X.head(5)
X.shape

In [ ]:
Y=tab['label']
print(Y.shape)
Y.head(5)

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y,test_size=0.25, random_state=0)

In [ ]:
print(Xtrain.shape)
print(Xtest.shape)
print(ytrain.shape)
print(ytest.shape)

In [ ]:
KKN = KNeighborsClassifier()
clf=KKN.fit(Xtrain,ytrain)

ypredit=clf.predict(Xtest)

accuracy_score(ytest,ypredit)